# MobiCount v1.0

In [1]:
# Autor : Bruno DATO
# Co-autor : Adrien LAMMOGLIA
# Date : 15/11/2025

## ➡️ Step 1 — Install dependencies

In [2]:
!pip install ultralytics opencv-python



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## ➡️ Step 2 — Set the project folder and the video name

In [3]:
PROJECT_FOLDER = "C:/Users/bruno/OneDrive/Documents/Repositories/MOBICOUNT/MobiCount"
VIDEO_NAME = "1191553-hd_1920_1080_25fps"

## ➡️ Step 3 — Set the parameters


In [4]:
CLASSES = [0, 1, 2, 3, 5, 7] # Filters results by class index. For example, classes=[0, 2, 3] only tracks persons, cars and motorcycles.

""" names:
  0: person
  1: bicycle
  2: car
  3: motorcycle
  5: bus
  7: truck
 """

REGION = [(1500, 0), (1500, 3000)]  # VERTICAL LINE
#REGION = [(0, 700), (1920, 700)]    # HORIZONTAL LINE
#REGION = [(860, 0), (860, 1080), (1060, 1080), (1060, 0)]  # VERTICAL RECTANGLE
#REGION = [(760, 0), (760, 1500), (1160, 1500), (1160, 0)]  # THIN VERTICAL RECTANGLE

SHOW_VIDEO = False

CONF = 0.3 # Sets the confidence threshold for detections; lower values allow more objects to be tracked but may include false positives.


## ➡️ Step 3 — Create Yolo instance and video writer

In [10]:
import cv2
import csv

from ultralytics import solutions

# Other parameters

VIDEO_FOLDER = PROJECT_FOLDER +"/Video/"
VIDEO_PATH = VIDEO_FOLDER + VIDEO_NAME + ".mp4"
RESULTS_PATH = PROJECT_FOLDER + "/Results/counts_"

# Open the video file

video_path = VIDEO_PATH
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"



# Video writer
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_writer = cv2.VideoWriter(RESULTS_PATH + VIDEO_NAME +".mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

print("Fps:",fps,"Size:",w,"x",h,"Total frames:",total_frames)

# Initialize object counter object
# https://docs.ultralytics.com/guides/object-counting/#real-world-applications
counter = solutions.ObjectCounter(
    show=SHOW_VIDEO,  # display the output
    region=REGION,  # List of points defining the counting region.
    model="yolo11n.pt",  # Path to Ultralytics YOLO Model File.
    classes=CLASSES,  # Filters results by class index. For example, classes=[0, 2, 3] only tracks the specified classes.
    tracker="bytetrack.yaml",  # Specifies the tracking algorithm to use, e.g., bytetrack.yaml (faster) or botsort.yaml.
    conf = CONF, # Sets the confidence threshold for detections; lower values allow more objects to be tracked but may include false positives.
    iou = 0.5, # Sets the Intersection over Union (IoU) threshold for filtering overlapping detections.
    verbose=False,
)

results = None

Fps: 25 Size: 1920 x 1080 Total frames: 213
Ultralytics Solutions:  {'source': None, 'model': 'yolo11n.pt', 'classes': [0, 1, 2, 3, 5, 7], 'show_conf': True, 'show_labels': True, 'region': [(1500, 0), (1500, 3000)], 'colormap': 21, 'show_in': True, 'show_out': True, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'figsize': (12.8, 7.2), 'blur_ratio': 0.5, 'vision_point': (20, 20), 'crop_dir': 'cropped-detections', 'json_file': None, 'line_width': 2, 'records': 5, 'fps': 30.0, 'max_hist': 5, 'meter_per_pixel': 0.05, 'max_speed': 120, 'show': False, 'iou': 0.5, 'conf': 0.3, 'device': None, 'max_det': 300, 'half': False, 'tracker': 'bytetrack.yaml', 'verbose': False, 'data': 'images'}


## ➡️ Step 4 — Process the Video

In [ ]:
# Process video
print("Processing Video...")
frameCounter = 0

while cap.isOpened():
    frameCounter = frameCounter + 1
    if frameCounter % (fps) == 0:
        ratio = frameCounter/total_frames
        print(str(round(ratio*100, 2)) + " % Frames processed")
    success, im0 = cap.read()

    if not success:
        print("100 % Frames processed")
        print("Video frame is empty or processing is complete.")
        break

    results = counter(im0)

    video_writer.write(results.plot_im)  # write the processed frame.


print("Results: " + str(results.classwise_count))



Processing Video...
11.737089201877934 % Frames processed
23.474178403755868 % Frames processed
35.2112676056338 % Frames processed
46.948356807511736 % Frames processed
58.68544600938967 % Frames processed
70.4225352112676 % Frames processed
82.15962441314554 % Frames processed
93.89671361502347 % Frames processed
Video frame is empty or processing is complete.
100 % Frames processed
Results: {'bus': {'IN': 0, 'OUT': 1}, 'car': {'IN': 0, 'OUT': 3}}


## ➡️ Step 5 — Write results (CSV)

In [7]:
with open(RESULTS_PATH + VIDEO_NAME + ".csv", "w", newline="") as f:
    fieldnames = ["TYPE", "IN", "OUT"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for vehicle, counts in results.classwise_count.items():
        row = {"TYPE": vehicle, **counts}
        writer.writerow(row)

print("Results available at " + RESULTS_PATH + VIDEO_NAME + ".csv")
print("Video available at " + RESULTS_PATH + VIDEO_NAME +".mp4")

cap.release()
video_writer.release()
cv2.destroyAllWindows()  # destroy all opened windows

Results available at C:/Users/bruno/OneDrive/Documents/Repositories/MOBICOUNT/MobiCount/Results/counts_1191553-hd_1920_1080_25fps.csv
Video available at C:/Users/bruno/OneDrive/Documents/Repositories/MOBICOUNT/MobiCount/Results/counts_1191553-hd_1920_1080_25fps.mp4
